In [1]:
import KNNclassy
import numpy as np
import random
import matplotlib.pyplot as plt
import seaborn as sns
import importlib
from sklearn.neighbors import NearestNeighbors
importlib.reload(KNNclassy)
import pandas as pd

# Functions 

In [2]:
def matrix_KNN_MDC(KNNdic, nneigh,eos,events):
    
    total_events = len(events[:,0])

    nplus_no = np.zeros(total_events)
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    
    for i in range(0,total_events):
        distance, closestn = KNNdic['knn'].model.kneighbors(events[i].reshape(1, -1),n_neighbors=nneigh,return_distance = True)
        if (distance.any() <= 0.001):
            print('event '+str(i)+' gives problems')
            print(distance[0])
        weighted_neighs = 1/distance[0]
        norm = np.sum(1/distance[0])
        labels = KNNdic['knn'].label_train[closestn][0].tolist()
        for ll in range(0,len(labels)): 
            labels[ll] = int(labels[ll])
        labels = np.array(labels)
        ind_0 = np.where(labels == 0)
        ind_1 = np.where(labels == 1)
        ind_2 = np.where(labels == 2)
    
        neighs_0 = weighted_neighs[ind_0]
        neighs_1 = weighted_neighs[ind_1]
        neighs_2 = weighted_neighs[ind_2]

        nplus_no[i] = np.sum(neighs_0)/norm
        nplus_NS[i] = np.sum(neighs_1)/norm
        nplus_REM[i] = np.sum(neighs_2)/norm
        
    return nplus_NS, nplus_REM, nplus_no

def matrix_KNN_test(KNNdic,nneigh,eos,nrandom,datatest,labels_test):
    
    total_events = len(datatest[:,0])
    nplus_NO = np.zeros(total_events)
    nplus_NS = np.zeros(total_events)
    nplus_REM = np.zeros(total_events)
    
    rnd_ev = random.sample(range(0,total_events), nrandom)
    print('random events: ',rnd_ev)
    
    total_events = len(rnd_ev)
    
    testing = datatest[rnd_ev]
    labelstest = labels_test[rnd_ev]
    
    for i in range(0,total_events):
        distance, closestn = KNNdic['knn'].model.kneighbors(testing[i].reshape(1, -1),n_neighbors=nneigh,return_distance = True)
        prediction = KNNdic['knn'].model.predict(testing[i].reshape(1, -1))
     
        if (distance.any() <= 0.001):
            print('event '+str(i)+' gives problems')
            print(distance[0])
        weighted_neighs = 1/distance[0]
        norm = np.sum(1/distance[0])
        labels = KNNdic['knn'].label_train[closestn][0].tolist()
        for ll in range(0,len(labels)): 
            labels[ll] = int(labels[ll])
        labels = np.array(labels)
        ind_0 = np.where(labels == 0)
        ind_1 = np.where(labels == 1)
        ind_2 = np.where(labels == 2)
    
        neighs_0 = weighted_neighs[ind_0]
        neighs_1 = weighted_neighs[ind_1]
        neighs_2 = weighted_neighs[ind_2]

        nplus_NO[i] = np.sum(neighs_0)/norm
        nplus_NS[i] = np.sum(neighs_1)/norm
        nplus_REM[i] = np.sum(neighs_2)/norm
        
        print('Nothing: ',nplus_NO[i])
        print('NS: ',nplus_NS[i])
        print('REM: ',nplus_REM[i])
        print('Labels neigh: ',labels)
        print('True label: ',labelstest[i])
        print('Prediction: ',prediction)
        print('*'*60)
        
    return 

### Definition of paths 

In [3]:
nneigh = [9]
pathClassy = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/"
pathData = "/Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/input/"
EOS = ["APR4_EPP", "BHF_BBB2", "H4", "HQC18", "KDE0V", "KDE0V1", "MPA1", "MS1_PP", "MS1B_PP", "RS", "SK255", "SK272", "SKI2", "SKI3", "SKI4", "SKI5", "SKI6", "SKMP", "SKOP", "SLy", "SLY2", "SLY9", "SLY230A"]
BayesFactor = [1.526, 1.555, 0.056, 1.422, 1.177, 1.283, 0.276, 0.001, 0.009, 0.176, 0.179, 0.159, 0.108, 0.107, 0.33, 0.025, 0.288, 0.29, 0.618, 1.0, 1.028, 0.37, 0.932]

In [4]:
EOSdic = {}
count = 0

## Loading KNN trained algorithms

In [5]:
for eos in EOS:
    
    print("Doing", eos)
    print('*'*60)

    for kk in nneigh:
        
        print("Nº of neighbors: ", kk)
        print('-'*40)

        KNN = KNNclassy.ClassificationKNN()
        KNN.load_train_dataset(pathData+eos+"/NS/merged_NS_"+eos+"_s300_f0d7.csv",pathData+eos+"/EMB/merged_EMB_"+eos+"_s300_f0d7.csv")
    
        KNN.loadModel(pathClassy+"optimal_models_final_fullO2/", "knn_3cat_grid_eos_"+eos)
        new_dic = {}
        new_dic['knn'] = KNN
        new_dic['bayes'] = BayesFactor[count]
        EOSdic[eos] = new_dic
    
        print('*'*60)
        
    count = count + 1
        

Doing APR4_EPP
************************************************************
Nº of neighbors:  9
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  198961
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final_fullO2/knn_3cat_grid_eos_APR4_EPP.joblib
************************************************************
Doing BHF_BBB2
************************************************************
Nº of neighbors:  9
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  193624
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final_fullO2/knn_3cat_gr

************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  199821
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final_fullO2/knn_3cat_grid_eos_SKI6.joblib
************************************************************
Doing SKMP
************************************************************
Nº of neighbors:  9
----------------------------------------
************************************************************
Loading training data...
Nº of features:  5
Nº of events for training:  197566
Datasets loaded!
************************************************************
loading  /Users/miquelmiravet/Projects/IPAM_LA/ML_group/KNN_miq/optimal_models_final_fullO2/knn_3cat_grid_eos_SKMP.joblib
************************************************************
Doing SKOP
******************************************************

## Read MDC11 dataset 

In [6]:
df = pd.read_csv(pathClassy+'MDC11.csv')
df.head()

,mass1,mass2,spin1z,spin2z,combined_snr,combined_far,pipeline,NS_SLy,EMB_SLy,NS_BHF_BBB2,...,NS_SKI3,EMB_SKI3,NS_SKI5,EMB_SKI5,NS_MPA1,EMB_MPA1,NS_MS1B_PP,EMB_MS1B_PP,NS_MS1_PP,EMB_MS1_PP
0,4.268033,3.634232,-0.913736,0.450203,18.991429,7.532970e-13,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
1,6.268945,2.588275,-0.146668,0.099328,19.285233,1.467042e-07,pycbc,1,0,1,...,1,0,1,0,1,0,1,0,1,0
2,12.247071,1.550140,0.370892,0.019994,19.721900,1.387861e-17,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,6.154887,2.612731,-0.016768,-0.532756,18.754279,1.323210e-11,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
4,21.466118,1.100672,0.726591,0.007167,15.509615,4.250779e-06,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0


In [7]:
df = df[df['pipeline'].str.contains('gstlal')]

columns = df[['mass1', 'mass2', 'spin1z', 'spin2z', 'combined_snr']]

# Convert the DataFrame to a Numpy array
events = columns.values
nplus_NS = []
nplus_REM = []
nplus_NO = []

In [8]:
labels_test = {}
for eos in EOS:
    columns = df[['NS_'+eos,'EMB_'+eos]]
    fracs = columns.values
    labels_test[eos] = fracs

## Compute neighbors for MDC11 data

In [9]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    npns,nprem,npno = matrix_KNN_MDC(EOSdic[eos],nneigh[0],eos,events)
    nplus_NS.append(npns)
    nplus_REM.append(nprem)
    nplus_NO.append(npno)
    print('#'*60)

Doing EOS:  APR4_EPP
############################################################
Doing EOS:  BHF_BBB2
############################################################
Doing EOS:  H4
############################################################
Doing EOS:  HQC18
############################################################
Doing EOS:  KDE0V
############################################################
Doing EOS:  KDE0V1
############################################################
Doing EOS:  MPA1
############################################################
Doing EOS:  MS1_PP
############################################################
Doing EOS:  MS1B_PP
############################################################
Doing EOS:  RS
############################################################
Doing EOS:  SK255
############################################################
Doing EOS:  SK272
############################################################
Doing EOS:  SKI2
################################

## Save fractions/number of neighbors of MDC11 data

In [10]:
data = pd.DataFrame({'event_ID':np.arange(0,len(events[:,0]))})
for i,eos in enumerate(EOS):
    data['HasNS_true'] = labels_test[eos][:,0]
    data['HasREM_true'] = labels_test[eos][:,1]
    data['f_0'] = nplus_NO[i]
    data['f_1'] = nplus_NS[i]
    data['f_2'] = nplus_REM[i]
    data.to_csv('matrix_KNN_MDC11_gstlal_EOS-'+eos+'.csv', index = False)   

## Testing

In [113]:
df = pd.read_csv(pathClassy+'MDC11.csv')
df.head()

,mass1,mass2,spin1z,spin2z,combined_snr,combined_far,pipeline,NS_SLy,EMB_SLy,NS_BHF_BBB2,...,NS_SKI3,EMB_SKI3,NS_SKI5,EMB_SKI5,NS_MPA1,EMB_MPA1,NS_MS1B_PP,EMB_MS1B_PP,NS_MS1_PP,EMB_MS1_PP
0,4.268033,3.634232,-0.913736,0.450203,18.991429,7.532970e-13,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
1,6.268945,2.588275,-0.146668,0.099328,19.285233,1.467042e-07,pycbc,1,0,1,...,1,0,1,0,1,0,1,0,1,0
2,12.247071,1.550140,0.370892,0.019994,19.721900,1.387861e-17,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
3,6.154887,2.612731,-0.016768,-0.532756,18.754279,1.323210e-11,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0
4,21.466118,1.100672,0.726591,0.007167,15.509615,4.250779e-06,spiir,1,0,1,...,1,0,1,0,1,0,1,0,1,0


In [11]:
labels_test = {}
for eos in EOS:
    columns = df[['NS_'+eos,'EMB_'+eos]]
    fracs = columns.values
    labels_test[eos] = fracs

In [91]:
for i in range(0,len(labels_test['APR4_EPP'][:])):
    print(labels_test['APR4_EPP'][i])

1
1
1
1
1
1
1
1
1
2
2
2
2
2
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
0
0
2
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
1
1
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
1
1
1
1
1
1
1
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
1
1
2
2
2
2
2
2
2
2
2
2
1
1
1
1
2
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
0
0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
0
0
0
0
0
1
0
0
2
2
2
2
2
2
0
0
0
0
0
2
2
2
1
2
2
2
2
2
2
0
0
0
0
0
0
1
2
2
2
2
0
0
0
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
1
1
1
0
0
0
0
0
0
0
2
2
0
0
0
1
0
0
0
2
2
2
2
2
2
2
2
1
1
2
1
1
1
1
1
0
0
0
2
2
2
2
0
1
1
1
2
2
2
2
2
2
2
2
1
1
1
1
1
2
2
2
1
1
1
1
1
1
1


1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
2
2
2
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
1
1
2
2
2
2
2
2
2
2
2
1
1
1
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
1
1
1
1
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
1
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
2
2
2
2
0
0
2
2
2
2
2
2
2
2
0
0
0
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
2
2
1
1
1
1
0
2
2
2
2
2
2
0
0
0
0
0
0


1
1
1
1
2
2
2
2
2
2
2
2
0
0
0
0
2
2
2
2
2
2
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
0
0
1
1
2
2
2
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
2
2
2
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
1
1
1
1
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
2
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
0
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
1
1
2
2
2
2
2
2
2
2
2
2
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0


1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
1
1
1
2
2
2
2
2
2
1
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
0
0
0
0
0
0
0
0
0
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
2
2
2
2
2
2
2
2
2


0
2
2
2
2
2
2
2
0
0
0
0
2
2
2
2
1
1
1
1
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
0
0
0
0
2
2
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
2
2
2
2
2
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
2
2
2
2
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
0
0
0
1
1
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
2
2
2
2
2
2
2
2
2
0
0
0
0


2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
2
2
2
2
2
2
2
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
1
1
1
1
1
1
1
1
1
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
2
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
1
1
2
2
2
2
2
2
2
2
2
2
2
2
0
0
2
2
2
2
0
0
0
0
0
0
0
0
0
2
2
1
1
0
0
2
2
2
2
2
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
0
0
0
0
2
2
2
2
2
2
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
1
1
0
0
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
0
2
2
2
2
2
2
2
2
0
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2


0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
2
2
2
2
2
2
1
0
0
0
0
0
1
1
1
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
1
2
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
1
1
1
1
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1
1
1
1
0
0
0
0
0
0
1
1
1
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
2
2
2
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
1
1
1
1
0
0
0
0
0
0
0
0
0
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
0
2
2
2
2
2
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
1
1
1
1
1
1
1
1
0
0
0
2
2
2
1
1
1
1
1
1
1
1
1
1
1
1
1
1
2
2
2
0
0
1
1
1
1
1
1
0
0
0
0
0
0
0
2
2
2
2
2
2
2
2
2
2
0
0
0
0
0
0
0
0
2
2
2
2
2
0
0
0
0
0
0
0
0
0
0
0
2
2
2
1
1
1
1
1
0
0
2
2
2
2
2
2
2
2
2
2
2
2
2
1
1
1


In [12]:
for eos in EOS:
    print('Doing EOS: ', eos)
    print('='*60)
    matrix_KNN_test(EOSdic[eos],nneigh[0],eos,3,events,labels_test[eos])

Doing EOS:  APR4_EPP
random events:  [210, 2058, 9146]
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
************************************************************
Nothing:  1.0
NS:  0.0
REM:  0.0
Labels neigh:  [0 0 0 0 0 0 0 0 0]
True label:  [0 0]
Prediction:  [0.]
************************************************************
Nothing:  0.7774645855517687
NS:  0.22253541444823108
REM:  0.0
Labels neigh:  [0 1 0 0 0 1 0 0 0]
True label:  [0 0]
Prediction:  [0.]
************************************************************
Doing EOS:  BHF_BBB2
random events:  [7902, 4053, 8659]
Nothing:  1.0
NS:  0.0
REM:  0.0
Labels neigh:  [0 0 0 0 0 0 0 0 0]
True label:  [0 0]
Prediction:  [0.]
************************************************************
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
************************************************************
Nothing:  0.0
NS:  0.0
REM: 

Prediction:  [2.]
************************************************************
Nothing:  1.0
NS:  0.0
REM:  0.0
Labels neigh:  [0 0 0 0 0 0 0 0 0]
True label:  [0 0]
Prediction:  [0.]
************************************************************
Doing EOS:  SLY9
random events:  [3671, 790, 9888]
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
************************************************************
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
************************************************************
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
************************************************************
Doing EOS:  SLY230A
random events:  [3571, 2930, 10424]
Nothing:  0.0
NS:  0.0
REM:  1.0
Labels neigh:  [2 2 2 2 2 2 2 2 2]
True label:  [1 1]
Prediction:  [2.]
*********************************************